<a href="https://colab.research.google.com/github/Vinayak-2003/Fake-News-Prediction-NLP-Machine-Learning/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Prediction

## Text Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import re                                 #regular expression
import nltk                               #Natural language toolkit
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [4]:
news_data = pd.read_csv('/content/drive/MyDrive/DataSet/Fake news.csv', engine='python', error_bad_lines=False)

<ipython-input-4-24cc90a757f2>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_data = pd.read_csv('/content/drive/MyDrive/DataSet/Fake news.csv', engine='python', error_bad_lines=False)
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 197

In [5]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news_data.shape

(20822, 5)

0 ----> real news

1 ----> fake news

In [7]:
news_data.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [8]:
#replacing all the missing values with NULL string
news_data = news_data.fillna('')

In [9]:
#merging author and text column
news_data['content'] = news_data['author'] + ' ' + news_data['title']

In [10]:
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [11]:
#seprating feature and target
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']

In [12]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20817  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20818  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20819  20797  Macy’s Is Said to Receive Takeover Approach by...   
20820  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20821  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [13]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20817    0
20818    0
20819    0
20820    1
20821    1
Name: label, Length: 20822, dtype: object


## Stemming
It is the process of converting the word to the root word

In [14]:
port_stem = PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)           #only accepting alphabets and removes all the punctuations
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  #convert all the words to their root words, ignoring the stopwords
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [16]:
news_data['content'] = news_data['content'].apply(stemming)

In [17]:
print(news_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20817    jerom hudson rapper trump poster child white s...
20818    benjamin hoffman n f l playoff schedul matchup...
20819    michael j de la merc rachel abram maci said re...
20820    alex ansari nato russia hold parallel exercis ...
20821                            david swanson keep f aliv
Name: content, Length: 20822, dtype: object


In [18]:
X = news_data['content'].values
Y = news_data['label'].values

In [19]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [20]:
print(Y)

['1' '0' '1' ... '0' '1' '1']


In [21]:
#converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [22]:
print(X)

  (0, 15695)	0.2848972510323515
  (0, 13478)	0.2560149172431211
  (0, 8912)	0.36364344795259246
  (0, 8633)	0.29217179802324905
  (0, 7695)	0.24789859828097843
  (0, 7008)	0.21878792690522433
  (0, 4975)	0.2333632865672138
  (0, 3794)	0.2705797813233836
  (0, 3602)	0.35994096489813754
  (0, 2960)	0.24689141014750257
  (0, 2484)	0.3676990592743697
  (0, 267)	0.27014778053672167
  (1, 16807)	0.30071934786077886
  (1, 6819)	0.1904866118084521
  (1, 5506)	0.7143145691090199
  (1, 3570)	0.2637458518155929
  (1, 2814)	0.19096625123288233
  (1, 2224)	0.3827200188088641
  (1, 1894)	0.15524631265972175
  (1, 1497)	0.2939921887587299
  (2, 15620)	0.4154491828226571
  (2, 9623)	0.4935010343775579
  (2, 5971)	0.3474726097377811
  (2, 5392)	0.3866575731579265
  (2, 3104)	0.4609666076640373
  :	:
  (20819, 13126)	0.24825540550304984
  (20819, 12348)	0.2726336911024517
  (20819, 12142)	0.2477854180074755
  (20819, 10310)	0.08040873089640542
  (20819, 9591)	0.17456730094721287
  (20819, 9521)	0.295416

splitting the data into training and testing data

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

Training the model

In [24]:
model = LogisticRegression()

In [25]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

In [26]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
print("Accuracy Score on training data: ", train_data_accuracy)

Accuracy Score on training data:  0.9924356126553401


In [28]:
#accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [29]:
print("Accuracy Score on test data: ", test_data_accuracy)

Accuracy Score on test data:  0.9815126050420168


Making the predictive system

In [30]:
X_news = X_test[11]

prediction = model.predict(X_news)
print(prediction)

if (prediction[0] == '1'):
  print("The news is Fake")
else:
  print("The news is Real")

['1']
The news is Fake


In [31]:
#for conforming
print(Y_test[11])

1
